In [ ]:
import requests
import pandas as pd
import matplotlib
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
driver = webdriver.Chrome()
driver.get('https://www.youtube.com/c/GenshinImpact/videos')

# Scrolling chrome page to end to get max results
# https://stackoverflow.com/a/64777192
height = driver.execute_script("return document.documentElement.scrollHeight")
previousHeight = -1

while previousHeight < height:
    previousHeight = height
    driver.execute_script(f'window.scrollTo(0,{height + 10000})')
    time.sleep(1)
    height = driver.execute_script("return document.documentElement.scrollHeight")

genshin_yt = []
count = 0
videos = driver.find_elements(By.CLASS_NAME, 'style-scope ytd-grid-video-renderer')
length_thumb = driver.find_elements(By.XPATH, 
            '//*[name() = "ytd-grid-video-renderer" and @class="style-scope ytd-grid-renderer"]')


# Looping to get information about each video. 
for video in videos: 
    title = video.find_element(By.XPATH, './/*[@id="video-title"]').text
    views = video.find_element(By.XPATH, './/*[@id="metadata-line"]/span[1]').text
    upload_date = video.find_element(By.XPATH, './/*[@id="metadata-line"]/span[2]').text

    vd = {
        'Title':title,
        'Views':views,
        'Upload Date':upload_date,
    }
    genshin_yt.append(vd)
    count += 1

# Getting length of videos
count2 = 0
duration_list = []
for i in length_thumb:
    duration = i.find_element(By.XPATH, './/span[contains(@class,"time-status")]').text
    duration_list.append(duration)
    count2 += 1

In [ ]:
genshin_df = pd.DataFrame(genshin_yt)
genshin_df['Duration'] = duration_list
genshin_df.head()

In [ ]:
# Converting to a csv file
genshin_df.to_csv(r'C:\Users\Roast\Desktop\genshin_yt.csv', index = False, header=True)

In [124]:
# Importing fixed csv file
genshin_df1 = pd.read_csv(r'C:\Users\roast\Downloads\genshin_yt.csv')
genshin_df1.head(5)

,Title,Views,Upload Date,Duration
0,"Character Teaser - ""Yelan: Inevitable Justice""...",1.6M views,14 hours ago,1:29
1,Version 2.7 Special Program｜Genshin Impact,1.1M views,5 days ago,36:06:00
2,"Version 2.7 ""Hidden Dreams in the Depths"" Trai...",1.8M views,5 days ago,3:48
3,Genshin Impact EP - The Shirasagi's Gentle Con...,692K views,1 month ago,4:19
4,"""Tsubaki in Thawing Snow"" Short Trailer | Gens...",4.1M views,1 month ago,3:12


In [125]:
genshin_df1['Title'] = genshin_df1['Title'].astype('string')
genshin_df1['Views'] = genshin_df1['Views'].astype('string')
genshin_df1['Upload Date'] = genshin_df1['Upload Date'].astype('string')
genshin_df1['Duration'] = genshin_df1['Duration'].astype('string')

In [126]:
# Removing "views" from Views column
for i in range(len(genshin_df1['Views'])):
    genshin_df1['Views'][i] = genshin_df1['Views'][i].replace(' views', "")

genshin_df1['Views'] = genshin_df1['Views'].astype('object')
for i in range(len(genshin_df1['Views'])):
    genshin_df1['Views'][i] = conv(genshin_df1['Views'][i])

In [128]:
genshin_df1['Views'] = genshin_df1['Views'].astype('int64')
print(genshin_df1.info())
genshin_df1.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226 entries, 0 to 225
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        226 non-null    string
 1   Views        226 non-null    int64 
 2   Upload Date  226 non-null    string
 3   Duration     226 non-null    string
dtypes: int64(1), string(3)
memory usage: 7.2 KB
None


,Title,Views,Upload Date,Duration
0,"Character Teaser - ""Yelan: Inevitable Justice""...",1600000,14 hours ago,1:29
1,Version 2.7 Special Program｜Genshin Impact,1100000,5 days ago,36:06:00
2,"Version 2.7 ""Hidden Dreams in the Depths"" Trai...",1800000,5 days ago,3:48
3,Genshin Impact EP - The Shirasagi's Gentle Con...,692000,1 month ago,4:19
4,"""Tsubaki in Thawing Snow"" Short Trailer | Gens...",4099999,1 month ago,3:12


### EDA Queries for Views

In [175]:
top_views = genshin_df1.sort_values(by='Views', ascending=False, axis=0, ignore_index= True)
top_views.iloc[:10] #Top 10 videos by view count. 

,Title,Views,Upload Date,Duration
0,Genshin Impact Story Teaser: We Will Be Reunit...,29000000,1 year ago,2:15
1,"Character Demo - ""Zhongli: The Listener"" | Gen...",27000000,1 year ago,1:55
2,"Collected Miscellany - ""Qiqi: Fortune-Preservi...",26000000,1 year ago,4:32
3,"New Character Demo - ""Eula: Flickering Candlel...",22000000,1 year ago,1:56
4,"""Pyro Chapter"" - Chef de Cuisine Xiangling｜Gen...",21000000,2 years ago,1:03
5,TGA 2021 Genshin Impact Entry Video｜Genshin Im...,19000000,5 months ago,1:41
6,"Character Demo - ""Hu Tao: Let the Living Bewar...",17000000,1 year ago,1:41
7,Teyvat Chapter Storyline Preview: Travail｜Gens...,16000000,1 year ago,4:19
8,"New Character Demo - ""Kaedehara Kazuha: Wander...",16000000,10 months ago,2:04
9,"Version 2.0 ""The Immovable God and the Eternal...",16000000,10 months ago,5:10


In [176]:
least_views = genshin_df1.sort_values(by='Views', ascending=True, axis=0, ignore_index= True)
least_views.iloc[:10] #Least 10 viewed videos by count

,Title,Views,Upload Date,Duration
0,#13 Knighthood Excellence｜Genshin Impact,105000,1 year ago,1:42
1,#11 A Tale of Two Dragons｜Genshin Impact,122000,1 year ago,2:16
2,Islands of the Lost and Forgotten - Disc 2: Be...,134000,1 month ago,44:03:00
3,#10 The Edge of the Prairie｜Genshin Impact,139000,1 year ago,1:20
4,#12 Rite of Battle｜Genshin Impact,149000,1 year ago,4:29
5,#07 A Day in Mondstadt｜Genshin Impact,155000,1 year ago,1:11
6,#09 Pure Sky｜Genshin Impact,161000,1 year ago,1:14
7,#15 The Wind Catcher from a Foreign Land｜Gensh...,179000,1 year ago,1:59
8,#06 Lone Sojourner｜Genshin Impact,198000,1 year ago,0:58
9,Islands of the Lost and Forgotten - Disc 1: Is...,222000,1 month ago,45:42:00


In [193]:
mean_views = genshin_df1['Views'].mean()
print('Mean: %.2f' % (mean_views)) 

Mean: 3905915.92


In [212]:
std_views = genshin_df1['Views'].std()

more_views_mean = len(genshin_df1.query('Views >= @mean_views'))
less_views_mean = len(genshin_df1.query('Views <= @mean_views'))

print('Overall percentage above mean: %.2f' % (more_views_mean/len(genshin_df1)*100))
print('Overall percentage below mean: %.2f' % (less_views_mean/len(genshin_df1)*100))


Overall percentage above mean: 32.74
Overall percentage below mean: 67.26


In [203]:
# Videos above/below the mean
print("Number of videos above mean: ", more_views_mean)
print("Number of videos below mean: ", less_views_mean)


Number of videos above mean:  74
Number of videos below mean:  152
